# Dependency Injection: 

In [1]:
abstract class LogSink : IDisposable {
    
    public abstract void WriteMessage(string message);
    
    public virtual void Dispose() { }
    
}

interface IFilenameManager {
    string NameNewLogFile();
}

class FileLogger : LogSink {
    
    string _Filename;

    // Now referencing the interface as a type instead of the FilenameManager class
    IFilenameManager _FilenameManager;

    public FileLogger(IFilenameManager filenameManager) {
        _FilenameManager = filenameManager;
        _Filename = _FilenameManager.NameNewLogFile();
        CreateNewFile(_Filename);
    }
    
    public void CreateNewFile(string filename) {
        // Create the log file
        _Filename = filename;
        display($"Created log file {filename}");
    }
    
    public override void WriteMessage(string message) {
        // write message to log file
        display("Wrote message to disk");
    }
    
    public override void Dispose() {
        // clean up and close the log file
        display($"Cleaned up and closed log file {_Filename}");
    }
    
}

class LoggerFilenameManager : IFilenameManager {
    
    public string NameNewLogFile() {
        var filename = DateTime.UtcNow.ToString("yyyyMMdd_HH") + ".log";
        display(filename);
        return filename;
    }
    
}

interface ILoggerFormatter {
    string FormatLogMessage(string rawMessage);
}

class NormalLoggerFormatter : ILoggerFormatter {
    
    public string FormatLogMessage(string rawMessage) {
        var msg = $"{DateTime.UtcNow.ToString("yyyyMMdd_HHmmss")} - {rawMessage}";
        display(msg);
        return msg;
    }
    
}

// Compose these objects into a single LoggerManager object that handles the operations for ther rest of our application
class LoggerManager : IDisposable {
    
    // Referencing the LogSink abstract class so that we can change our abstract interaction with LogSinks later
    LogSink _Logger;
    
    public LoggerManager(LogSink sink) {
        _Logger = sink;
    }
    
    public void LogMessage(string message) {
        LogMessage(message, new NormalLoggerFormatter());
    }
    
    
    // Referencing the ILoggerFormatter so that we can inject the desired formatter
    public void LogMessage(string message, ILoggerFormatter formatter) {
        var newMessage = formatter.FormatLogMessage(message);
        _Logger.WriteMessage(newMessage);
    }
    
    public void Dispose() {
        _Logger.Dispose();
    }
    
}

var fileNameManager = new LoggerFilenameManager();
var fileSink = new FileLogger(fileNameManager);
var logger = new LoggerManager(fileSink);
logger.LogMessage("Error detected");
logger.Dispose();


The below script needs to be able to find the current output cell; this is an easy method to get it.

20230602_15.log

Created log file 20230602_15.log

20230602_150009 - Error detected

Wrote message to disk

Cleaned up and closed log file 20230602_15.log

In [2]:
using System;
using System.IO;
using System.Collections;
using System.Collections.Generic;
using System.Linq;

public class App
{
    // Excercise the Iterator and show that it's more
    // performant.
    public static void Main()
    {
        TestStreamReaderEnumerable();
        Console.WriteLine("---");
        TestReadingFile();
    }

    public static void TestStreamReaderEnumerable()
    {
        // Check the memory before the iterator is used.
        long memoryBefore = GC.GetTotalMemory(true);
      IEnumerable<String> stringsFound;
        // Open a file with the StreamReaderEnumerable and check for a string.
      try {
         stringsFound =
               from line in new StreamReaderEnumerable(@"c:\temp\tempFile.txt")
               where line.Contains("string to search for")
               select line;
         Console.WriteLine("Found: " + stringsFound.Count());
      }
      catch (FileNotFoundException) {
         Console.WriteLine(@"This example requires a file named C:\temp\tempFile.txt.");
         return;
      }

        // Check the memory after the iterator and output it to the console.
        long memoryAfter = GC.GetTotalMemory(false);
        Console.WriteLine("Memory Used With Iterator = \t"
            + string.Format(((memoryAfter - memoryBefore) / 1000).ToString(), "n") + "kb");
    }

    public static void TestReadingFile()
    {
        long memoryBefore = GC.GetTotalMemory(true);
      StreamReader sr;
      try {
         sr = File.OpenText("c:\\temp\\tempFile.txt");
      }
      catch (FileNotFoundException) {
         Console.WriteLine(@"This example requires a file named C:\temp\tempFile.txt.");
         return;
      }

        // Add the file contents to a generic list of strings.
        List<string> fileContents = new List<string>();
        while (!sr.EndOfStream) {
            fileContents.Add(sr.ReadLine());
        }

        // Check for the string.
        var stringsFound =
            from line in fileContents
            where line.Contains("string to search for")
            select line;

        sr.Close();
        Console.WriteLine("Found: " + stringsFound.Count());

        // Check the memory after when the iterator is not used, and output it to the console.
        long memoryAfter = GC.GetTotalMemory(false);
        Console.WriteLine("Memory Used Without Iterator = \t" +
            string.Format(((memoryAfter - memoryBefore) / 1000).ToString(), "n") + "kb");
    }
}

// A custom class that implements IEnumerable(T). When you implement IEnumerable(T),
// you must also implement IEnumerable and IEnumerator(T).
public class StreamReaderEnumerable : IEnumerable<string>
{
    private string _filePath;
    public StreamReaderEnumerable(string filePath)
    {
        _filePath = filePath;
    }

    // Must implement GetEnumerator, which returns a new StreamReaderEnumerator.
    public IEnumerator<string> GetEnumerator()
    {
        return new StreamReaderEnumerator(_filePath);
    }

    // Must also implement IEnumerable.GetEnumerator, but implement as a private method.
    private IEnumerator GetEnumerator1()
    {
        return this.GetEnumerator();
    }
    IEnumerator IEnumerable.GetEnumerator()
    {
        return GetEnumerator1();
    }
}

// When you implement IEnumerable(T), you must also implement IEnumerator(T),
// which will walk through the contents of the file one line at a time.
// Implementing IEnumerator(T) requires that you implement IEnumerator and IDisposable.
public class StreamReaderEnumerator : IEnumerator<string>
{
    private StreamReader _sr;
    public StreamReaderEnumerator(string filePath)
    {
        _sr = new StreamReader(filePath);
    }

    private string _current;
    // Implement the IEnumerator(T).Current publicly, but implement
    // IEnumerator.Current, which is also required, privately.
    public string Current
    {

        get
        {
            if (_sr == null || _current == null)
            {
                throw new InvalidOperationException();
            }

            return _current;
        }
    }

    private object Current1
    {

        get { return this.Current; }
    }

    object IEnumerator.Current
    {
        get { return Current1; }
    }

    // Implement MoveNext and Reset, which are required by IEnumerator.
    public bool MoveNext()
    {
        _current = _sr.ReadLine();
        if (_current == null)
            return false;
        return true;
    }

    public void Reset()
    {
        _sr.DiscardBufferedData();
        _sr.BaseStream.Seek(0, SeekOrigin.Begin);
        _current = null;
    }

    // Implement IDisposable, which is also implemented by IEnumerator(T).
    private bool disposedValue = false;
    public void Dispose()
    {
        Dispose(disposing: true);
        GC.SuppressFinalize(this);
    }

    protected virtual void Dispose(bool disposing)
    {
        if (!this.disposedValue)
        {
            if (disposing)
            {
                // Dispose of managed resources.
            }
            _current = null;
            if (_sr != null) {
               _sr.Close();
               _sr.Dispose();
            }
        }

        this.disposedValue = true;
    }

     ~StreamReaderEnumerator()
    {
        Dispose(disposing: false);
    }
}
// This example displays output similar to the following:
//       Found: 2
//       Memory Used With Iterator =     33kb
//       ---
//       Found: 2
//       Memory Used Without Iterator =  206kb


(11,24): warning CS7022: The entry point of the program is global code; ignoring 'App.Main()' entry point.

